# Oskar tracker for transcriptomes
- Authors: Savandara BESSE & Leo BLONDEL
- Creation: 10-05-2019
- Modification: 10-27-2019
______________________


In [1]:
from Bio import SeqIO
import os, progressbar

## Step 1.A : Create protein sequences from TSA nucleotide sequences
    - Unzip files
    - Rename files 
    - Translation (with clean translation end)

In [26]:
def build_TSA_inputs(TSA_path, protein_path):
    protein_list = []
    bar = progressbar.ProgressBar()
    if not os.path.isdir(protein_path):
        os.mkdir(protein_path)
        
#     os.system('gunzip {}/*'.format(TSA_path))
#     os.system('rename \'s/fsa_nt$/fasta/\' {}/*.fsa_nt'.format(TSA_path))
    
    for TSA in bar(os.listdir(TSA_path)):
        INPUT = os.path.join(TSA_path, TSA)
        OUTPUT = os.path.join(protein_path, 'translated_{}'.format(TSA))
        os.system('transeq -sequence {} -frame 6 -trim -clean -outseq {}'.format(INPUT,OUTPUT))    
        protein_list.append(OUTPUT)
    return protein_list

In [7]:
%%time 
TSA_path = '/media/savvy/DATA2/savvy/EXTAVOUR/SOURCES/TSA/TSA_INPUTS'
protein_path = '/media/savvy/DATA2/savvy/EXTAVOUR/SOURCES/TSA/TSA_PROTEIN'
translated_TSA = build_TSA_inputs(TSA_path, protein_path)

100% (1114 of 1114) |####################| Elapsed Time: 0:49:34 Time:  0:49:34


CPU times: user 3.08 s, sys: 3.6 s, total: 6.67 s
Wall time: 57min 56s


In [27]:
## Crustacean analysis 2017
%%time 
TSA_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/TSA_INPUTS'
protein_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/TSA_PROTEIN'
translated_TSA = build_TSA_inputs(TSA_path, protein_path)

100% (168 of 168) |######################| Elapsed Time: 0:18:03 Time:  0:18:03


CPU times: user 340 ms, sys: 830 ms, total: 1.17 s
Wall time: 18min 3s


## Step 1.B: Collect GCF protein
    - Copy GCF protein fasta files
    - Gunzip them

In [16]:
def reach_Proteins(currentFolder):
    for gFile in os.listdir(currentFolder):
        if '_protein.faa.gz' in gFile :
            return gFile
        
def build_GCF_inputs(genome_path, res_path):
    for folder in os.listdir(genome_path): 
        if 'GCF' in folder :
            currentFolder = os.path.join(genome_path, folder)
            proteome = reach_Proteins(currentFolder)
            target = os.path.join(genome_path, currentFolder, proteome)
            os.system('cp {} {}'.format(target, res_path))

In [17]:
%%time 
genome_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/Genomes'
protein_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/GCF_PROTEIN'
build_GCF_inputs(genome_path, protein_path)
os.system('gunzip {}/*'.format(protein_path))

## Step 1.C: Collect GCA protein
    - Prepare Augustus inputs
    - Run Augustus annotations on cluster
    - Collect GCA gff files 
    - Convert them into protein fasta files 

In [103]:
def reach_Genes(currentFolder):
    all_genomic_files = [ gFile for gFile in os.listdir(currentFolder) if '_genomic.fna.gz' in gFile ]
    if len(all_genomic_files) > 1 :
        for gFile in all_genomic_files :
            if ('cds' in gFile) or ('rna' in gFile):
                pass
            else :
                return gFile
    else :
        return all_genomic_files[0]
        
def build_GCA_inputs(genome_path, res_path):
    for folder in os.listdir(genome_path): 
        if 'GCA' in folder :
            currentFolder = os.path.join(genome_path, folder)
            proteome = reach_Genes(currentFolder)
            target = os.path.join(genome_path, currentFolder, proteome)
            os.system('cp {} {}'.format(target, res_path))

In [ ]:
### Create inputs for Augustus
%%time 
genome_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/Genomes'
res_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/GCA_INPUTS'
build_GCA_inputs(genome_path, res_path)

### /!\ After collecting GFF annotation files from Cluster
    - Run getAnnoFasta.pl
    - Move proteins files in GCA_protein

## Step 2 : Track Oskar

#### TSA

In [9]:
%%time 
lotus_model = '../Data/Oskar_hmm/LOTUS_CONSENSUS.hmm'
osk_model = '../Data/Oskar_hmm/OSK_CONSENSUS.hmm'
protein_path = '/media/savvy/DATA2/savvy/EXTAVOUR/SOURCES/TSA/TSA_PROTEIN'
result_path = '/media/savvy/DATA2/savvy/EXTAVOUR/SOURCES/TSA/TSA_RESULT'
os.system('python3 execute_hmmsearch.py -a TSA -p {} -l {} -o {} -r {}'.format(protein_path, lotus_model, osk_model, result_path))

CPU times: user 36.6 ms, sys: 23.9 ms, total: 60.5 ms
Wall time: 27min 14s


In [29]:
%%time 
## Crustacean analysis 
lotus_model = '../Data/Oskar_hmm/LOTUS_CONSENSUS.hmm'
osk_model = '../Data/Oskar_hmm/OSK_CONSENSUS.hmm'
protein_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/TSA_PROTEIN'
result_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/TSA_RESULT'
os.system('python3 execute_hmmsearch.py -a TSA -p {} -l {} -o {} -r {}'.format(protein_path, lotus_model, osk_model, result_path))

CPU times: user 12.2 ms, sys: 10.9 ms, total: 23 ms
Wall time: 6min 37s


#### GCF

In [19]:
%%time 
lotus_model = '../Data/Oskar_hmm/LOTUS_CONSENSUS.hmm'
osk_model = '../Data/Oskar_hmm/OSK_CONSENSUS.hmm'
protein_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/GCF_PROTEIN'
result_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/GCF_RESULT'
os.system('python3 execute_hmmsearch.py -a GCF -p {} -l {} -o {} -r {}'.format(protein_path, lotus_model, osk_model, result_path))

CPU times: user 2.87 ms, sys: 3.64 ms, total: 6.51 ms
Wall time: 32.2 s


#### GCA

In [2]:
%%time 
lotus_model = '../Data/Oskar_hmm/LOTUS_CONSENSUS.hmm'
osk_model = '../Data/Oskar_hmm/OSK_CONSENSUS.hmm'
protein_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/GCA_PROTEIN'
result_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/GCA_RESULT'
os.system('python3 execute_hmmsearch.py -a GCA -p {} -l {} -o {} -r {}'.format(protein_path, lotus_model, osk_model, result_path))

CPU times: user 6.68 ms, sys: 0 ns, total: 6.68 ms
Wall time: 1min 17s


## STEP 3 : Collect Oskar candidates

In [5]:
from Bio import SearchIO
import numpy as np
import pandas as pd

def collect_Table(TSA, HMM):
    for DOM in HMM :
        if TSA in DOM :
            return DOM
        
def collect_Hits(result_path, hmmerTable):
    f = open(os.path.join(result_path, hmmerTable))
    tmp = [ line for line in f.readlines() if '#' not in line ]
    f.close()
    if len(tmp) != 0 : 
        return [ hit.id for hit in SearchIO.read(hmmerTable, 'hmmer3-tab') if hit.evalue <= 0.05 ]
    return []
    

def oskar_analysis(ID):
    lotus_res = os.path.join(result_path, collect_Table(ID, LOTUS))
    osk_res = os.path.join(result_path, collect_Table(ID, OSK))

    lotus_hits = collect_Hits(result_path, lotus_res)
    osk_hits = collect_Hits(result_path, osk_res)
    oskar_hits = list(set(lotus_hits) & set(osk_hits))
    
    if len(oskar_hits) == 0 :
        oskar_hits = 'None'
    else : 
        oskar_hits = ','.join(list(set(lotus_hits) & set(osk_hits)))
    if len(lotus_hits) == 0 :
        lotus_hits = 'None'
    else :
        lotus_hits = ','.join(list(lotus_hits))
    if len(osk_hits) == 0:
        osk_hits = 'None'
    else : 
        osk_hits = ','.join(list(osk_hits))

    return [ID, lotus_hits, osk_hits, oskar_hits ]
        
PGC_mode = {
    'Diplura':'Induction',
    'Archaeognatha':'Induction',
    'Zygentoma':'Induction',
    'Odonata':'Induction',
    'Ephemeroptera':'Induction',
    'Zoraptera':'Induction',
    'Dermaptera':'Induction',
    'Plecoptera':'Induction',
    'Orthoptera':'Induction',
    'Mantophasmatodea':'Induction',
    'Grylloblattodea':'Induction',
    'Embioptera':'Induction',
    'Phasmatodea':'Induction',
    'Mantodea':'Induction',
    'Blattodea':'Induction',
    'Isoptera':'Induction',
    'Thysanoptera':'Induction',
    'Hemiptera':'Induction',
    'Phthiraptera':'Induction',
    'Psocoptera':'Induction',
    'Hymenoptera':'Inheritance',
    'Raphidioptera':'Inheritance',
    'Megaloptera':'Inheritance',
    'Neuroptera':'Inheritance',
    'Strepsiptera':'Inheritance',
    'Coleoptera':'Inheritance',
    'Trichoptera':'Inheritance',
    'Lepidoptera':'Inheritance',
    'Siphonaptera':'Inheritance',
    'Mecoptera':'Inheritance',
    'Diptera':'Inheritance'
}

def set_augustus_model(order, table):
    if order == 'Diplura' :
        hmm_order = 'frankliniella_occidentalis'
    elif order == 'Archaeognatha':
        hmm_order = 'frankliniella_occidentalis'
    elif order == 'Odonata' :
        hmm_order = 'zootermopsis_nevadensis'
    elif order == 'Ephemeroptera' :
        hmm_order = 'frankliniella_occidentalis'
    elif order == 'Plecoptera' :
        hmm_order = 'zootermopsis_nevadensis'
    elif order == 'Orthoptera' :
        hmm_order = 'zootermopsis_nevadensis'
    elif order == 'Phasmatodea' :
        hmm_order = 'zootermopsis_nevadensis'
    elif order == 'Blattodea':
        hmm_order = 'zootermopsis_nevadensis'
    elif order == 'Thysanoptera' :
        hmm_order = 'frankliniella_occidentalis'
    elif order == 'Hemiptera' :
        hmm_order = 'bemisia_tabaci'
    elif order == 'Phthiraptera' :
        hmm_order = 'pediculus_humanus'
    elif order == 'Hymenoptera' :
        hmm_order = 'apis_mellifera'
    elif order == 'Strepsiptera' :
        hmm_order = 'tribolium_castaneum'
    elif order == 'Coleoptera' :
        hmm_order = 'tribolium_castaneum'
    elif order == 'Trichoptera' :
        hmm_order = 'papilio_xuthus'
    elif order == 'Lepidoptera' :
        hmm_order = 'papilio_xuthus'
    elif order == 'Siphonaptera' :
        hmm_order = 'ctenophalides_felis'
    elif order == 'Diptera' :
        family = table[table['order_name'] == order ]['family_name']
        if 'Culicidae' in family :
            hmm_order = 'aedes'
        elif 'Pteromalidae' in family :
            hmm_order = 'nasonia'
        else :
            hmm_order = 'fly'
    return hmm_order

def set_germ_cell_formation(x):
    return PGC_mode[x]

#### TSA

In [313]:
%%time
tsa_path = '../Data/01_Oskar_identification/hmmsearch_raw_results/TSA'
IDs = list(set([ TSA.split('_')[0] for TSA in os.listdir('../Data/01_Oskar_identification/hmmsearch_raw_results/TSA') ]))
LOTUS = [ domain for domain in sorted(os.listdir(tsa_path)) if 'lotus' in domain ]
OSK = [ domain for domain in sorted(os.listdir(tsa_path)) if 'osk' in domain ]
TSA_OSKAR = list(map(oskar_analysis, IDs))

CPU times: user 1.5 s, sys: 28 ms, total: 1.53 s
Wall time: 1.53 s


In [314]:
TMP = pd.DataFrame(TSA_OSKAR, columns=['tsa_abrv', 'lotus_hits', 'osk_hits', 'oskar_hits'])
TSA = pd.read_csv('../Data/01_Oskar_identification/2019/transcriptome_insect_database.csv')

TSA['pgc_mode'] = TSA['order_name'].apply(set_germ_cell_formation)
TSA['tsa_abrv'] = [ '{}{}'.format(TSA['tsa_id'][i][:5], TSA['tsa_id'][i].split('.')[1]) for i in range(len(TSA)) ]

TSA = TSA.merge(TMP, on='tsa_abrv')
TSA.to_csv('../Data/01_Oskar_identification/oskar_tracker_results/TSA/tsa_oskar_results.csv', index=False, na_rep='None')

In [31]:
%%time
## Curstacean analysis 
tsa_path = '../Data/01_Oskar_identification/hmmsearch_raw_results/TSA_crustacea'
IDs = list(set([ TSA.split('_')[0] for TSA in os.listdir('../Data/01_Oskar_identification/hmmsearch_raw_results/TSA_crustacea') ]))
LOTUS = [ domain for domain in sorted(os.listdir(tsa_path)) if 'lotus' in domain ]
OSK = [ domain for domain in sorted(os.listdir(tsa_path)) if 'osk' in domain ]
TSA_OSKAR = list(map(oskar_analysis, IDs))

CPU times: user 327 ms, sys: 37.6 ms, total: 364 ms
Wall time: 433 ms


In [50]:
def formatTSA(x):
    x = x.replace('.1', '')
    return x

In [53]:
TMP = pd.DataFrame(TSA_OSKAR, columns=['TSA', 'lotus_hits', 'osk_hits', 'oskar_hits'])
TMP['TSA'] = TMP['TSA'].apply(formatTSA)
TSA = pd.read_csv('../Data/01_Oskar_identification/2017/transcriptome_crustacean_database.csv')
# TSA['pgc_mode'] = TSA['order_name'].apply(set_germ_cell_formation)
TSA = TSA.merge(TMP, on='TSA')
TSA.to_csv('../Data/01_Oskar_identification/oskar_tracker_results/TSA_crustacea/tsa_oskar_results.csv', index=False, na_rep='None')

#### GCF 

In [30]:
gcf_path = '../Data/01_Oskar_identification/hmmsearch_raw_results/GCF'
IDs = list(set([ '{}_{}'.format(GCF.split('_')[0], GCF.split('_')[1]) for GCF in os.listdir(gcf_path) ]))
LOTUS = [ domain for domain in sorted(os.listdir(gcf_path)) if 'lotus' in domain ]
OSK = [ domain for domain in sorted(os.listdir(gcf_path)) if 'osk' in domain ]


In [49]:
GCF_OSKAR = list(map(oskar_analysis, IDs))
TMP = pd.DataFrame(GCF_OSKAR, columns=['genome_id', 'lotus_hits', 'osk_hits', 'oskar_hits'])

GCF = pd.read_csv('../Data/01_Oskar_identification/2019/genome_insect_database.csv')
GCF['pgc_mode'] = GCF['order_name'].apply(set_germ_cell_formation)

GCF = GCF.merge(TMP, on='genome_id')
GCF.to_csv('../Data/01_Oskar_identification/oskar_tracker_results/GCF/gcf_oskar_results.csv', index=False, na_rep='None')

#### GCA

In [3]:
gca_path = '../Data/01_Oskar_identification/hmmsearch_raw_results/GCA'
IDs = list(set([ '{}_{}'.format(GCA.split('_')[0], GCA.split('_')[1]) for GCA in os.listdir(gca_path) ]))
LOTUS = [ domain for domain in sorted(os.listdir(gca_path)) if 'lotus' in domain ]
OSK = [ domain for domain in sorted(os.listdir(gca_path)) if 'osk' in domain ]

In [6]:
GCA_OSKAR = list(map(oskar_analysis, IDs))
TMP = pd.DataFrame(GCA_OSKAR, columns=['genome_id', 'lotus_hits', 'osk_hits', 'oskar_hits'])

GCA = pd.read_csv('../Data/01_Oskar_identification/2019/genome_insect_database.csv')
GCA['pgc_mode'] = GCA['order_name'].apply(set_germ_cell_formation)
GCA['augustus_model'] = GCA['order_name'].apply(set_augustus_model, args=(GCA,))
GCA = GCA.merge(TMP, on='genome_id')
GCA.to_csv('../Data/01_Oskar_identification/oskar_tracker_results/GCA/gca_oskar_results.csv', index=False, na_rep='None')

## Step 4: Collect candidates

In [7]:
def collect_genome(x, table):
    return table[table['oskar_hits'].str.contains(x)]['genome_id'].values[0]

def retrieveSeq(seqID, dataType, fastaFile):
    for seqRecord in SeqIO.parse(fastaFile, 'fasta'):
        if seqID in seqRecord.id :
            if 'TSA' in dataType :
                seqRecord.seq = format_Seq(seqRecord.seq)
            return seqRecord
        
def first_AA(seq):
    for i in range(len(seq)):
        if 'M' in seq[i]:
            return i

def format_Seq(seq):
    return seq[first_AA(seq):]

def save_oskar(TABLE, protein_path, dataType, result_file):
    OSKAR_SEQ = [] 
    OSKAR_HITS = [ ID for LIST in TABLE[TABLE['oskar_hits'] != 'None']['oskar_hits'] for ID in LIST.split(',') ]
    bar = progressbar.ProgressBar()
    for SEQ in bar(OSKAR_HITS): 
        if 'TSA' in dataType :
            ID = SEQ[:4]
        else :
            ID = collect_genome(SEQ, TABLE)
        for PROTEOME in os.listdir(protein_path):
            if ID in PROTEOME :
                FASTA = os.path.join(protein_path, PROTEOME)
                OSKAR_SEQ.append(retrieveSeq(SEQ, dataType, FASTA))
    SeqIO.write(OSKAR_SEQ, result_file, 'fasta')        

In [161]:
protein_path = '/media/savvy/DATA2/savvy/EXTAVOUR/SOURCES/TSA/TSA_PROTEIN'
result_file = '../Data/01_Oskar_identification/oskar_tracker_results/TSA/tsa_oskar_candidates.fasta'
save_oskar(TSA, protein_path, 'TSA', result_file)

100% (326 of 326) |######################| Elapsed Time: 0:08:21 Time:  0:08:21


In [63]:
protein_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/GCF_PROTEIN'
result_file = '../Data/01_Oskar_identification/oskar_tracker_results/GCF/gcf_oskar_candidates.fasta'
save_oskar(GCF, protein_path, 'GCF', result_file)

100% (98 of 98) |########################| Elapsed Time: 0:00:08 Time:  0:00:08


In [8]:
protein_path = '/media/lblondel/5D1FA0DA2BE76E76/savy/SOURCES/GCA_PROTEIN'
result_file = '../Data/01_Oskar_identification/oskar_tracker_results/GCA/gca_oskar_candidates.fasta'
save_oskar(GCA, protein_path, 'GCA', result_file)

100% (98 of 98) |########################| Elapsed Time: 0:00:08 Time:  0:00:08


## Step 5: Validate candidates

In [194]:
tsa_candidates = '../Data/01_Oskar_identification/oskar_tracker_results/TSA/tsa_oskar_candidates.fasta'
oskar_model = '../Data/Oskar_hmm/OSKAR_CONSENSUS.hmm'
oskar_validation_results = '../Data/01_Oskar_identification/oskar_tracker_results/TSA/final_tsa_oskar_search.txt'
os.system('hmmsearch --cpu 8 --tblout {} {} {}'.format(oskar_validation_results, oskar_model, tsa_candidates))

In [61]:
gcf_candidates = '../Data/01_Oskar_identification/oskar_tracker_results/GCF/gcf_oskar_candidates.fasta'
oskar_model = '../Data/Oskar_hmm/OSKAR_CONSENSUS.hmm'
oskar_validation_results = '../Data/01_Oskar_identification/oskar_tracker_results/GCF/final_gcf_oskar_search.txt'
os.system('hmmsearch --cpu 8 --tblout {} {} {}'.format(oskar_validation_results, oskar_model, gcf_candidates))

0

In [9]:
gca_candidates = '../Data/01_Oskar_identification/oskar_tracker_results/GCA/gca_oskar_candidates.fasta'
oskar_model = '../Data/Oskar_hmm/OSKAR_CONSENSUS.hmm'
oskar_validation_results = '../Data/01_Oskar_identification/oskar_tracker_results/GCA/final_gca_oskar_search.txt'
os.system('hmmsearch --cpu 8 --tblout {} {} {}'.format(oskar_validation_results, oskar_model, gca_candidates))

0

## Step 6: Oskar duplicate and isoform detection and filtering

In [1]:
import networkx as nx
from Bio import SeqIO
from Bio import SearchIO
from Bio import Align
from Bio import AlignIO
from Bio import Seq
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
TSA_results = pd.read_csv('../Data/01_Oskar_identification/oskar_tracker_results/TSA/tsa_oskar_results.csv')
TSA_sequences = '../Data/01_Oskar_identification/oskar_tracker_results/TSA/tsa_oskar_candidates.fasta'
hmmerTable = '../Data/01_Oskar_identification/oskar_tracker_results/TSA/final_tsa_oskar_search.txt'
TSA_hmmer = SearchIO.read(hmmerTable, 'hmmer3-tab')
TSA_filtered_outpath = '../Data/01_Oskar_identification/oskar_tracker_results/TSA/tsa_oskar_filtered.fasta'

In [3]:
GCF_results = pd.read_csv('../Data/01_Oskar_identification/oskar_tracker_results/GCF/gcf_oskar_results.csv')
GCF_sequences = '../Data/01_Oskar_identification/oskar_tracker_results/GCF/gcf_oskar_candidates.fasta'
GCF_hmmer = SearchIO.read('../Data/01_Oskar_identification/oskar_tracker_results/GCF/final_gcf_oskar_search.txt', 'hmmer3-tab')
GCF_filtered_outpath = '../Data/01_Oskar_identification/oskar_tracker_results/GCF/gcf_oskar_filtered.fasta'

In [4]:
# REMOVE THIS !
GCA_results = GCF_results

In [5]:
# GCA_results = pd.read_csv('../Data/01_Oskar_identification/oskar_tracker_results/GCA/gca_oskar_results.csv')
# GCA_sequences = '../Data/01_Oskar_identification/oskar_tracker_results/GCA/gca_oskar_candidates.fasta'
# GCA_hmmer = SearchIO.read('../Data/01_Oskar_identification/oskar_tracker_results/GCA/final_gca_oskar_search.txt', 'hmmer3-tab')
# GCA_filtered_outpath = '../Data/01_Oskar_identification/oskar_tracker_results/GCA/gca_oskar_filtered.fasta'

In [6]:
filtered_outpath = '../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.fasta'

In [7]:
# Return the Hamming distance between string1 and string2.
# string1 and string2 should be the same length.
def hamming_distance(seq1, seq2): 
    assert(len(seq1) == len(seq2)) 
    # Start with a distance of zero, and count up
    distance = 0.0
    # Loop over the indices of the string
    L = len(seq1)
    for i in range(L):
        # Add 1 to the distance if these two characters are not equal
        if seq1[i] != seq2[i]:
            distance += 1
    # Return the final count of differences
    return 1 - distance/L
#     return distance

def group_sequences(sources):
    # Sources definition: [('ID', DataFrame), ('ID', Dataframe)....]
    
    sequences = {}
    for source in sources:
        for taxid, oskar_ids in source[1][source[1]['oskar_hits'] != "None"][['tax_id', 'oskar_hits']].values:
            if taxid not in sequences:
                sequences[taxid] = []
            sequences[taxid] += [(source[0], i.strip()) for i in oskar_ids.split(',')]
    return sequences

def make_tmp_seq_groups(sequences, sources):
    # sources ->   [('ID', fasta_path), ('ID', fasta_path)....]
    name2seq = {}

    for source in sources:
        fasta = [s for s in SeqIO.parse(source[1], 'fasta')]
        for seq in fasta:
            if seq.name in name2seq:
                print("ERROR SEQUENCE ALREADY EXISTS !")
                return False
            name2seq[seq.name] = seq

    tmp = './tmp/oskars'
    if not os.path.isdir('tmp'):
        os.mkdir('tmp')
    if not os.path.isdir(tmp):
        os.mkdir(tmp)
    sequences_groups = []
    for taxid in sequences:
        if len(sequences[taxid]) > 1:
            tmpseqs = []
            for origin, seq in sequences[taxid]:
                s = name2seq[seq]
                s.description += '|' + origin
                tmpseqs.append(s)
            outpath = os.path.join(tmp, '{}.fasta'.format(taxid))
            sequences_groups.append(outpath)
            f = open(outpath, 'w')
            SeqIO.write(tmpseqs, f, 'fasta')
    return sequences_groups
            
def muscle_align(inpath):
    "Command line wrapper for muscle, muscle needs to be in your path !"
    outpath = inpath.replace('fasta', 'aligned.fasta')
    cmd = 'muscle -in {} -out {}'.format(inpath, outpath)
    os.system(cmd)
    return outpath
    
def align_sequences(sequences_groups):
    outpaths = []
    for path in tqdm(sequences_groups):
        outpath = muscle_align(path)
        outpaths.append(outpath)
    return outpaths

def trim_alignement(alignement):
    res = []
    for i in range(alignement.get_alignment_length()):
        col = alignement[:, i]
        if '-' not in col:
            res.append([s for s in col])
    res = np.array(res).T
    for i in range(len(alignement)):
        seq = ''.join(list(res[i]))
        alignement[i].seq = seq
    return alignement
    
def make_network(alignement_path):
    alignement = AlignIO.read(alignement_path, "fasta")
    trimed = trim_alignement(alignement)
    hammings = np.zeros((len(trimed), len(trimed)))
    nodes = []
    for i in range(len(trimed)):
        nodes.append(trimed[i].name)
        for j in range(i+1, len(trimed)):
            seq1 = trimed[i].seq
            seq2 = trimed[j].seq
            hammings[i][j] = hamming_distance(seq1, seq2)
    M = np.maximum( hammings, hammings.transpose() )
    G = nx.from_numpy_matrix(hammings)
    for i in G.nodes():
        G.node[i]['seq_id'] = nodes[i]
    return G, nodes
   
def find_clusters(alignements_path, threshold=0.9):
    clusters = {}
    for alignement_path in alignements_path:
        taxid = int(alignement_path.split('/')[-1].split('.')[0])
        clusters[taxid] = []
        G, nodes = make_network(alignement_path)
        toremove = []
        for n1,n2 in G.edges():
            if G.edges[n1, n2]['weight'] < threshold:
                toremove.append((n1, n2))
        for n1, n2 in toremove:
            G.remove_edge(n1, n2)
        for cc in nx.connected_components(G):
            tmp = []
            for i in cc:
                tmp.append(nodes[i])
            clusters[taxid].append(tmp)
    return clusters

# def filter_sequences(clusters, TSA_sequences_path, TSA_hmmer_table, GCF_sequences_path, GCF_hmmer_table, uniq_sequences):
def filter_sequences(clusters, sources, uniq_sequences):
    # [('TSA', TSA_seq_path, TSA_hmmer_path), ('GCF', GCF_seq .......
    result = []
    sequences = {}
    scores = {}

    # source element is 
    # ('ID', sequence_path, hmmer_hit_object)
    for source in sources:
        handle = SeqIO.parse(source[1], 'fasta')
        for s in handle:
            sequences[s.name] = (source[0], s)
        for hit in source[2]:
            scores[hit.id] = hit.evalue
    
    for s in uniq_sequences:
        result.append(sequences[s[1]])
            
    for taxid in clusters:
        for cluster in clusters[taxid]:
            tmp_score = []
            for seq_id in cluster:
                tmp_score.append([scores[seq_id], sequences[seq_id]])
            best_seq = sorted(tmp_score, key=lambda x: x[0])
            result.append(best_seq[0][1])
    return result

def clean_sequences(sequences):
    cleaned = []
    for seq in sequences:
        tmp = ""
        pos = 0
        for l in seq.seq:
            if l != 'X':
                tmp += l
            elif pos < 550:
                tmp += l
            else:
                break
            pos += 1
        seq.seq = Seq.Seq(tmp)
        cleaned.append(seq)
    return cleaned

def decorate_sequences(sequences, TSA_metadatas, GCF_metadatas, GCA_metadatas=None):
    results = []
    for s in sequences:
        tag = s[0]
        seq = s[1] 
        if tag == 'TSA':
            TSA_ID = seq.name[0:6]
            metadata = TSA_metadatas[TSA_metadatas['tsa_abrv'] == TSA_ID][['species', 'family_name', 'order_name']].values[0]
        if tag == 'GCF':
            if len(seq.description.split('|')) != 4:
                GCF_specie = seq.description.split('[')[-1].split(']')[0]
                metadata = GCF_metadatas[GCF_metadatas['species'] == GCF_specie][['species', 'family_name', 'order_name']].values[0]
            else:
                metadata = seq.description.split('|')
        if tag == 'GCA':
            if len(seq.description.split('|')) != 4:
                GCA_specie = seq.description.split('[')[-1].split(']')[0]
                metadata = GCA_metadatas[GCA_metadatas['species'] == GCA_specie][['species', 'family_name', 'order_name']].values[0]
            else:
                metadata = seq.description.split('|')
        seq.description = "{}|{}|{}|{}".format(metadata[0], metadata[1], metadata[2], tag)
        results.append(seq)
    return results

In [8]:
# We first group the sequences by unique taxa
sequences = group_sequences([
    ('TSA', TSA_results),
    ('GCF', GCF_results),
#     ('GCA', GCA_results),
])

uniq_sequences = [sequences[k][0] for k in sequences if len(sequences[k]) == 1]

In [9]:
# Then we create fasta files for each taxa, only if this taxa has more than one sequence
sequences_groups = make_tmp_seq_groups(
    sequences,
    [('TSA', TSA_sequences),
     ('GCF', GCF_sequences),
#      ('GCA', GCA_sequences)
    ])

In [10]:
# We use Muscle to align those sequences (from the fasta files we just created)
alignements_path = align_sequences(sequences_groups)

In [11]:
# We use graph theory (simple edge removal by threshold) to find sequences cluster within the sequence group
clusters = find_clusters(alignements_path, threshold=0.80)

In [12]:
# We filter the sequences keeping the best E-value sequence for each cluster group,
# and add back the sequences that were unique already
filtered_sequences = filter_sequences(clusters, [
                                                ('TSA', TSA_sequences, TSA_hmmer),
                                                ('GCF', GCF_sequences, GCF_hmmer),
#                                                 ('GCA', GCA_sequences, GCA_hmmer)
                                                ], 
                                      uniq_sequences)

In [13]:
# We decorate the sequences for further processing with species, family and order name as well as a datatype tag "TSA"
decorated_sequences = decorate_sequences(filtered_sequences, TSA_results, GCF_results ,GCA_results)


In [14]:
cleaned_sequences = clean_sequences(decorated_sequences)

In [15]:
# We save our sequence group
f = open(filtered_outpath, 'w')
SeqIO.write(cleaned_sequences, f, 'fasta')
f.close()

In [16]:
handle = SeqIO.parse(TSA_sequences, 'fasta')
total_seq = 0
for s in handle:
    total_seq += 1

tsa_rem = total_seq
handle = SeqIO.parse(GCF_sequences, 'fasta')
for s in handle:
    total_seq += 1
print("Starting sequences")
print("TSA:", tsa_rem)
print("GCF:", total_seq-tsa_rem)
print("Total:", total_seq)


    
handle = SeqIO.parse(filtered_outpath, 'fasta')
filtered_seq = 0
tsa_keep = 0
gcf_keep = 0
for s in handle:
    filtered_seq += 1
    tag = s.description.split('|')[-1]
    if tag == 'TSA':
        tsa_keep += 1
    elif tag == 'GCF':
        gcf_keep += 1

print("The algorithm filtered out {} sequences".format(total_seq-filtered_seq))
print("Final sequence count is {}. TSA:{} | GCF:{}".format(filtered_seq, tsa_keep, gcf_keep))

Starting sequences
TSA: 326
GCF: 98
Total: 424
The algorithm filtered out 122 sequences
Final sequence count is 302. TSA:227 | GCF:75


In [17]:
# Redoing the same process but with only one source type at a time for count purposes

# Doing TSA
sequences = group_sequences([
    ('TSA', TSA_results),
])
uniq_sequences = [sequences[k][0] for k in sequences if len(sequences[k]) == 1]
sequences_groups = make_tmp_seq_groups(
    sequences,
    [('TSA', TSA_sequences),
    ])
alignements_path = align_sequences(sequences_groups)
clusters = find_clusters(alignements_path, threshold=0.80)
filtered_sequences = filter_sequences(clusters, [
                                                ('TSA', TSA_sequences, TSA_hmmer),
                                                ], 
                                      uniq_sequences)
decorated_sequences = decorate_sequences(filtered_sequences, TSA_results, GCF_results, GCA_results)
cleaned_sequences = clean_sequences(decorated_sequences)

# We save our sequence group
f = open(TSA_filtered_outpath, 'w')
SeqIO.write(cleaned_sequences, f, 'fasta')
f.close()
print("Sequences saved !")


Sequences saved !


In [18]:
handle = SeqIO.parse(TSA_sequences, 'fasta')
total_seq = 0
for s in handle:
    total_seq += 1

tsa_rem = total_seq
handle = SeqIO.parse(GCF_sequences, 'fasta')
for s in handle:
    total_seq += 1
print("Starting sequences")
print("TSA:", tsa_rem)
print("GCF:", total_seq-tsa_rem)
print("Total:", total_seq)


    
handle = SeqIO.parse(filtered_outpath, 'fasta')
filtered_seq = 0
tsa_keep = 0
gcf_keep = 0
for s in handle:
    filtered_seq += 1
    tag = s.description.split('|')[-1]
    if tag == 'TSA':
        tsa_keep += 1
    elif tag == 'GCF':
        gcf_keep += 1

print("The algorithm filtered out {} sequences".format(total_seq-filtered_seq))
print("Final sequence count is {}. TSA:{} | GCF:{}".format(filtered_seq, tsa_keep, gcf_keep))

Starting sequences
TSA: 326
GCF: 98
Total: 424
The algorithm filtered out 122 sequences
Final sequence count is 302. TSA:227 | GCF:75


In [19]:
# Doing GCF
sequences = group_sequences([
    ('GCF', GCF_results),
])
uniq_sequences = [sequences[k][0] for k in sequences if len(sequences[k]) == 1]
sequences_groups = make_tmp_seq_groups(
    sequences,
    [
     ('GCF', GCF_sequences),
    ])
alignements_path = align_sequences(sequences_groups)
clusters = find_clusters(alignements_path, threshold=0.80)
filtered_sequences = filter_sequences(clusters, [
                                                ('GCF', GCF_sequences, GCF_hmmer),
                                                ], 
                                      uniq_sequences)
decorated_sequences = decorate_sequences(filtered_sequences, TSA_results, GCF_results ,GCA_results)
cleaned_sequences = clean_sequences(decorated_sequences)

# We save our sequence group
f = open(GCF_filtered_outpath, 'w')
SeqIO.write(cleaned_sequences, f, 'fasta')
f.close()
print("Sequences saved !")


Sequences saved !


In [22]:
len(cleaned_sequences)

78

In [ ]:
# Doing GCA
sequences = group_sequences([
    ('GCA', GCA_results),
])
uniq_sequences = [sequences[k][0] for k in sequences if len(sequences[k]) == 1]
sequences_groups = make_tmp_seq_groups(
    sequences,
    [
     ('GCA', GCA_sequences)
    ])
alignements_path = align_sequences(sequences_groups)
clusters = find_clusters(alignements_path, threshold=0.80)
filtered_sequences = filter_sequences(clusters, [
                                                ('GCA', GCA_sequences, GCA_hmmer)
                                                ], 
                                      uniq_sequences)
decorated_sequences = decorate_sequences(filtered_sequences, TSA_results, GCF_results ,GCA_results)
cleaned_sequences = clean_sequences(decorated_sequences)

# We save our sequence group
f = open(GCA_filtered_outpath, 'w')
SeqIO.write(cleaned_sequences, f, 'fasta')
f.close()

In [48]:
GCF_results

,genome_id,tax_id,species,family_id,family_name,order_id,order_name,bioproject,biosample,download_status,lotus_hits,osk_hits,oskar_hits,pgc_mode
0,GCF_003285875.2,47314,Drosophila novamexicana,7214.0,Drosophilidae,7147,Diptera,PRJNA475270,SAMN09383009,True,"XP_030572142.1,XP_030558430.1,XP_030573785.1,X...","XP_030572142.1,XP_030573402.1",XP_030572142.1,Inheritance
1,GCF_006496715.1,7160,Aedes albopictus,7157.0,Culicidae,7147,Diptera,PRJNA530512,SAMN11317331,True,"XP_019540707.2,XP_019558301.2,XP_029723149.1,X...","XP_019558301.2,XP_019540707.2,XP_029710371.1,X...","XP_019558301.2,XP_019540707.2",Inheritance
2,GCF_003285725.1,7225,Scaptodrosophila lebanonensis,7214.0,Drosophilidae,7147,Diptera,PRJNA475270,SAMN09383169,True,"XP_030374856.1,XP_030388337.1,XP_030388338.1,X...",XP_030374856.1,XP_030374856.1,Inheritance
3,GCF_003285905.1,7224,Drosophila hydei,7214.0,Drosophilidae,7147,Diptera,PRJNA475270,SAMN09383128,True,"XP_023173869.2,XP_023162524.2,XP_023167456.1,X...",XP_023173869.2,XP_023173869.2,Inheritance
4,GCF_005508785.1,7029,Acyrthosiphon pisum,27482.0,Aphididae,7524,Hemiptera,PRJNA496478,SAMN10253041,True,"XP_008185199.1,XP_016663178.1,XP_029346403.1,X...",None,None,Induction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,GCF_000475195.1,121845,Diaphorina citri,1585420.0,Liviidae,7524,Hemiptera,PRJNA29447,SAMN00100712,True,"XP_026676435.1,XP_026676086.1,XP_008488072.1,X...","XP_008484229.1,XP_026687642.1,XP_026681044.1",None,Induction
128,GCF_000472105.1,28584,Drosophila suzukii,7214.0,Drosophilidae,7147,Diptera,PRJNA213258,SAMN02953868,True,"XP_016941563.1,XP_016928705.1,XP_016928704.1,X...","XP_016941563.1,XP_016941489.1",XP_016941563.1,Inheritance
129,GCF_000469605.1,7462,Apis dorsata,7458.0,Apidae,7399,Hymenoptera,PRJNA174631,SAMN02954476,True,"XP_006612511.1,XP_006614786.1,XP_006609155.1,X...",None,None,Inheritance
130,GCF_000005925.1,7260,Drosophila willistoni,7214.0,Drosophilidae,7147,Diptera,PRJNA12664,SAMN02953653,True,"XP_023034501.1,XP_002070281.2,XP_023033702.1,X...","XP_002070281.2,XP_002065327.1",XP_002070281.2,Inheritance


## Align the final result with hmmalign and refine with muscle

In [183]:
!hmmalign ../Data/Oskar_hmm/OSKAR_CONSENSUS.hmm ../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.fasta > ../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.hmmaligned.sto 

In [184]:
from Bio import SeqIO

i = open('../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.hmmaligned.sto')
o = open('../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.hmmaligned.fasta', 'w')

SeqIO.convert(i, 'stockholm', o, 'fasta')

302

In [185]:
!muscle -in ../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.hmmaligned.fasta -out ../Data/01_Oskar_identification/oskar_tracker_results/oskar_filtered.aligned.fasta -refine


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

00:00:46    41 MB(-3%)  Iter   1  100.00%  Refine biparts
00:01:25    42 MB(-3%)  Iter   2  100.00%  Refine biparts
00:02:02    44 MB(-3%)  Iter   3  100.00%  Refine biparts
00:02:12    44 MB(-3%)  Iter   4  100.00%  Refine biparts
00:02:12    44 MB(-3%)  Iter   4  100.00%  Refine biparts
